In [3]:
import numpy as np
import torch
import torch.nn as nn

In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70],
                  [74,66,43],
                  [91,87,65],
                  [88,134,59],
                  [101,44,37],
                  [68,96,71],
                  [73,66,44],
                  [92,87,64],
                  [87,135,57],
                  [103,43,36],
                  [68,97,70]],
                 dtype='float32')
inputs.shape

(15, 3)

In [7]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [8]:
inputs= torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [9]:
inputs


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

Dataset and DataLoader
We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [10]:
from torch.utils.data import TensorDataset

In [11]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [12]:
from torch.utils.data import DataLoader

In [13]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [14]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  97.,  70.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [103.,  43.,  36.],
        [ 87., 135.,  57.]])
tensor([[102., 120.],
        [103., 119.],
        [ 57.,  69.],
        [ 20.,  38.],
        [118., 134.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [48]:
# Define model
model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2263, -0.1990, -0.2200],
        [-0.3630, -0.5396, -0.1057]], requires_grad=True)
Parameter containing:
tensor([ 0.3787, -0.4825], requires_grad=True)


In [49]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2263, -0.1990, -0.2200],
         [-0.3630, -0.5396, -0.1057]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3787, -0.4825], requires_grad=True)]

In [50]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)
def model(x):
    return x @ w.t() + b

tensor([[-1.1195, -0.5985,  0.3576],
        [-0.9711,  0.8209, -0.5307]], requires_grad=True)
tensor([ 2.3031, -0.7583], requires_grad=True)


In [51]:

preds4=model(inputs)
preds4

tensor([[-104.1441,  -39.4632],
        [-129.3544,  -50.8472],
        [-154.5551,   -6.0168],
        [-124.3897,  -84.1418],
        [-107.3685,  -26.1006],
        [-104.6651,  -41.2552],
        [-128.3982,  -52.1988],
        [-155.3169,   -7.5185],
        [-123.8688,  -82.3499],
        [-105.8914,  -25.6602],
        [-103.1880,  -40.8148],
        [-129.8753,  -52.6391],
        [-155.5112,   -4.6652],
        [-125.8668,  -84.5822],
        [-106.8475,  -24.3086]], grad_fn=<AddBackward0>)

Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [52]:
# Import nn.functional
import torch.nn.functional as F

In [21]:
# Define loss function
loss_fn=F.mse_loss

In [53]:
loss= loss_fn(model(inputs),targets)
print(loss)

tensor(30034.3047, grad_fn=<MseLossBackward>)


Optimizer
Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [54]:
# Define optimizer
opt = torch.optim.SGD(model_nn.parameters(),lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:
Generate predictions
Calculate the loss
Compute gradients w.r.t the weights and biases
Adjust the weights by subtracting a small quantity proportional to the gradient
Reset the gradients to zero

In [55]:
# Utility function to train the model
def fit(num_epochs, model_nn, loss_fn, opt, train_dl):
     # Repeat for given number of epochs
        for epoch in range(num_epochs):
             # Train with batches of data
            for xb, yb in train_dl:
                # 1. Generate predictions
                pred = model(xb)
                # 2. Calculate loss
                loss = loss_fn(pred, yb)
                # 3. Compute gradients
                loss.backward()
                # 4. Update parameters using gradients
                opt.step()
                # 5. Reset the gradients to zero
                opt.zero_grad()
            # Print the progress
            if (epoch+1)%10 == 0:
                print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs, loss))

In [56]:
fit (100,model_nn, loss_fn, opt, train_dl)

Epoch [10/100], Loss:27605.3809
Epoch [20/100], Loss:27605.3809
Epoch [30/100], Loss:29649.3027
Epoch [40/100], Loss:29769.9570
Epoch [50/100], Loss:23976.6914
Epoch [60/100], Loss:21660.9922
Epoch [70/100], Loss:33181.6172
Epoch [80/100], Loss:35364.2422
Epoch [90/100], Loss:41487.8203
Epoch [100/100], Loss:35848.5430


In [57]:
# Generate predictions
preds= model(inputs)
preds

tensor([[-104.1441,  -39.4632],
        [-129.3544,  -50.8472],
        [-154.5551,   -6.0168],
        [-124.3897,  -84.1418],
        [-107.3685,  -26.1006],
        [-104.6651,  -41.2552],
        [-128.3982,  -52.1988],
        [-155.3169,   -7.5185],
        [-123.8688,  -82.3499],
        [-105.8914,  -25.6602],
        [-103.1880,  -40.8148],
        [-129.8753,  -52.6391],
        [-155.5112,   -4.6652],
        [-125.8668,  -84.5822],
        [-106.8475,  -24.3086]], grad_fn=<AddBackward0>)

In [58]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Wrong!! I can't compare input and targets